In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_Full/models/B5R2_15AB_Full_set1_AddNewDataset.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image_13122564/'
dataframe = pd.read_csv( '/home/yupaporn/CSV_file/Train_Dataset_CCA_5300.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/CSV_file/Test_Dataset_CCA_1328.csv')
base_dir0 = '/media/tohn/SSD/Images/Image_13122564/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv( '/home/yupaporn/CSV_file/Train_Dataset_CCA_5300.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(5300, 25)
(5300, 25)
Normal:  (3434, 25)
Abnormal:  (1866, 25)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Path Full,Path Crop,Views,fold,...,height,Rleft,Rtop,Rwidth,Rheight,Sub_class_New,tag_AjNit,tag_AjWan,Test150,Spilt
0,0,0,AB01_15,P3,P31,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,...,584.0,0.036832,0.107041,0.712707,0.649402,AB01,NaN,NaN,False,Train
1,1,1,AB01_15,P1,P1,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,...,614.0,0.143898,0.107041,0.595628,0.683267,AB01,NaN,NaN,False,Train
2,2,2,AB01_15,P2,P2,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,...,451.0,0.245009,0.113017,0.497278,0.501992,AB01,NaN,NaN,False,Train
3,3,3,AB01_15,P4,P42,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,...,611.0,0.155235,0.122977,0.718412,0.679283,AB01,NaN,NaN,False,Train
4,4,4,AB01_15,P4,P41,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,...,519.0,0.152057,0.162817,0.608229,0.577689,AB01,NaN,NaN,False,Train


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class_New',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 5300 validated image filenames belonging to 20 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'AB13', 15: 'AB14', 16: 'AB15', 17: 'AB16', 18: 'AB17', 19: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.16045688,  0.90175402, -0.09513459, ...,  0.68435663,
         0.73863649, -0.03870155],
       [ 0.03317273,  0.72681105, -0.12135908, ...,  0.75368392,
         0.54861253, -0.09152438],
       [-0.10289238,  0.64623249, -0.0730648 , ...,  0.66232318,
         0.43310973,  0.03302493],
       ...,
       [-0.05265403,  0.14941755, -0.10982462, ..., -0.10911254,
        -0.07509345,  0.05570506],
       [ 0.72169232, -0.10897655, -0.05721068, ...,  0.15257822,
         0.04068905,  0.16558231],
       [-0.1192117 , -0.15322812, -0.15370311, ..., -0.13396068,
         0.29971904, -0.0095561 ]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.160457,0.901754,-0.095135,-0.015717,-0.079120,-0.164944,0.374934,0.134212,-0.174954,-0.182251,...,-0.144220,0.108863,-0.119568,-0.068679,0.070255,0.153183,0.584165,0.684357,0.738636,-0.038702
1,0.033173,0.726811,-0.121359,-0.011929,0.007653,-0.183892,0.487158,0.051630,-0.115517,-0.155750,...,-0.145722,0.220647,-0.209329,-0.035593,-0.193423,-0.024856,0.388101,0.753684,0.548613,-0.091524
2,-0.102892,0.646232,-0.073065,0.087293,0.193535,-0.172595,0.607716,-0.075694,-0.146095,-0.194684,...,-0.148544,0.153514,-0.149293,0.013496,-0.189901,-0.178211,0.683081,0.662323,0.433110,0.033025
3,0.038968,0.479386,0.079918,-0.154050,0.150541,-0.003634,0.443206,-0.045163,-0.150804,-0.169418,...,-0.136825,-0.010169,-0.050947,0.010724,-0.121204,0.286660,0.246771,0.294574,0.299174,-0.129578
4,0.059911,0.207812,-0.180650,-0.087309,0.174356,-0.183723,0.369602,-0.116254,-0.143960,-0.106292,...,-0.139600,-0.055609,-0.035358,0.004183,-0.033036,-0.121227,0.310855,0.928555,0.781129,-0.149236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,-0.046794,0.049360,-0.085686,0.769817,0.049823,-0.029628,-0.159061,-0.172973,-0.049274,-0.031438,...,-0.008784,0.729277,0.609092,-0.122424,0.430098,-0.121540,-0.157882,-0.131303,0.491071,-0.049946
5296,0.412856,-0.098408,-0.165534,0.765477,-0.164004,0.791747,-0.115439,-0.047477,-0.163614,-0.015490,...,0.408860,0.945650,1.006355,0.093669,0.239596,-0.051432,-0.134834,0.133897,-0.030719,-0.027994
5297,-0.052654,0.149418,-0.109825,0.072172,0.052574,0.878487,-0.173367,0.014987,0.116442,0.072380,...,0.122423,0.923802,0.073751,0.197070,-0.073291,-0.093540,-0.133054,-0.109113,-0.075093,0.055705
5298,0.721692,-0.108977,-0.057211,0.700875,-0.111954,1.018799,-0.138699,-0.017943,-0.130606,-0.050650,...,0.569477,0.589168,1.066675,0.096270,0.063291,0.065891,-0.129623,0.152578,0.040689,0.165582


In [11]:
dataframe.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Case', 'Abs Position', 'Sub Position',
       'Class', 'Path Full', 'Path Crop', 'Views', 'fold', '15AB',
       'originalImage', 'left', 'top', 'width', 'height', 'Rleft', 'Rtop',
       'Rwidth', 'Rheight', 'Sub_class_New', 'tag_AjNit', 'tag_AjWan',
       'Test150', 'Spilt', 'category', 'Prob'],
      dtype='object')

In [14]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class_New'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class_New'] = d

In [15]:
a = dataframe['Case'].tolist()
dataset['Case'] = a 
b = dataframe['15AB'].tolist()
dataset['15AB'] = b

In [16]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2044,2045,2046,2047,Case,15AB,Class,SubPosition,Views,Sub_class_New
0,0.160457,0.901754,-0.095135,-0.015717,-0.079120,-0.164944,0.374934,0.134212,-0.174954,-0.182251,...,0.584165,0.684357,0.738636,-0.038702,AB01_15,MildFattyLiver,Abnormal,P31,FP-B,AB01
1,0.033173,0.726811,-0.121359,-0.011929,0.007653,-0.183892,0.487158,0.051630,-0.115517,-0.155750,...,0.388101,0.753684,0.548613,-0.091524,AB01_15,MildFattyLiver,Abnormal,P1,FP-A,AB01
2,-0.102892,0.646232,-0.073065,0.087293,0.193535,-0.172595,0.607716,-0.075694,-0.146095,-0.194684,...,0.683081,0.662323,0.433110,0.033025,AB01_15,MildFattyLiver,Abnormal,P2,FP-A,AB01
3,0.038968,0.479386,0.079918,-0.154050,0.150541,-0.003634,0.443206,-0.045163,-0.150804,-0.169418,...,0.246771,0.294574,0.299174,-0.129578,AB01_15,MildFattyLiver,Abnormal,P42,FP-B,AB01
4,0.059911,0.207812,-0.180650,-0.087309,0.174356,-0.183723,0.369602,-0.116254,-0.143960,-0.106292,...,0.310855,0.928555,0.781129,-0.149236,AB01_15,MildFattyLiver,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,-0.046794,0.049360,-0.085686,0.769817,0.049823,-0.029628,-0.159061,-0.172973,-0.049274,-0.031438,...,-0.157882,-0.131303,0.491071,-0.049946,AB17_05,LiverCyst,Abnormal,NaN,NaN,AB17
5296,0.412856,-0.098408,-0.165534,0.765477,-0.164004,0.791747,-0.115439,-0.047477,-0.163614,-0.015490,...,-0.134834,0.133897,-0.030719,-0.027994,AB17_08,LiverCyst,Abnormal,NaN,NaN,AB17
5297,-0.052654,0.149418,-0.109825,0.072172,0.052574,0.878487,-0.173367,0.014987,0.116442,0.072380,...,-0.133054,-0.109113,-0.075093,0.055705,AB17_20,LiverCyst,Abnormal,NaN,NaN,AB17
5298,0.721692,-0.108977,-0.057211,0.700875,-0.111954,1.018799,-0.138699,-0.017943,-0.130606,-0.050650,...,-0.129623,0.152578,0.040689,0.165582,AB17_09,LiverCyst,Abnormal,NaN,NaN,AB17


In [20]:
os.chdir( '/home/yupaporn/CSV_file/')
testdf = dataset
testdf.to_csv('FVTrain_20Class.csv')